# core

> This module contains all the core functions used in the library.

In [ ]:
#| default_exp core

In [ ]:
#| export
import logging
import os

from datasets import concatenate_datasets, Dataset
from rich.logging import RichHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(RichHandler(rich_tracebacks=True))

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from datasets import load_dataset
from squeakily.filter import check_char_repetition, check_flagged_words, minhash_dedup
from squeakily.clean import remove_empty_lines, normalize_whitespace

In [ ]:
#| export
class Pipeline:
    """
    A pipeline is a collection of datasources and their associated transformations to be run.
    """
    def __init__(
        self,
        datasources # The datasources to be run
    ):
        self.datasources = datasources
    
    def run(
        self,
        global_filters=[], # Filters to be run at the dataset level rather than the example level
        global_cleaners=[], # Cleaners to be run at the dataset level rather than the example level
        cleaning_first=False, # Whether to run the cleaning transformations first
        globals_first=False, # Whether to run the global transformations first
    ):
        """
        Run the pipeline.
        """
        for datasource in self.datasources:
            dataset = datasource["dataset"]
            column = datasource["columns"][0]
            logger.info(f"Running datasource: {dataset.builder_name}")
            if cleaning_first:
                for c in datasource["cleaners"]:
                    logger.info(f"Running cleaner: {c.__name__} on {column}")
                    dataset = dataset.map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )
                for f in datasource["filters"]:
                    logger.info(f"Running filter: {f.__name__} on {column}")
                    dataset = dataset.filter(lambda x: f(x[column]))
            else:
                for f in datasource["filters"]:
                    logger.info(f"Running filter: {f.__name__} on {column}")
                    dataset = dataset.filter(lambda x: f(x[column]))
                for c in datasource["cleaners"]:
                    logger.info(f"Running cleaner: {c.__name__} on {column}")
                    dataset = dataset.map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )
        
        if global_filters:
            # concatenate all datasets
            datasets = [d["dataset"] for d in self.datasources]
            global_column = self.datasources[0]["columns"][0]
            global_dataset = concatenate_datasets(datasets)

            # Add a column representing the original dataset name
            md = []
            for d in datasets:
                md.extend([d.builder_name] * len(d))
            meta_data = Dataset.from_dict({"meta_data": md})
            global_dataset_with_meta = concatenate_datasets([global_dataset, meta_data], axis=1)

            # Run the global filters
            for f in global_filters:
                logger.info(f"Running global filter: {f.__name__}")
                global_dataset_with_meta = f(global_dataset_with_meta, global_column)

            # Split the dataset back up
            for i, dataset in enumerate(datasets):
                self.datasources[i]["dataset"] = global_dataset_with_meta.filter(lambda x: x["meta_data"] == dataset.builder_name)

In [ ]:
#|echo: true
show_doc(Pipeline.run)

---

[source](https://github.com/CarperAI/squeakily/blob/main/squeakily/core.py#L28){target="_blank" style="float:right; font-size:smaller"}

### Pipeline.run

>      Pipeline.run (global_filters=[], global_cleaners=[],
>                    cleaning_first=False, globals_first=False)

Run the pipeline.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| global_filters | list | [] | Filters to be run at the dataset level rather than the example level |
| global_cleaners | list | [] | Cleaners to be run at the dataset level rather than the example level |
| cleaning_first | bool | False | Whether to run the cleaning transformations first |
| globals_first | bool | False | Whether to run the global transformations first |

In [ ]:
ds = load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")
logger.info(f"Original dataset size: {len(ds)}")
datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
    },
    # ...
]

global_filters = [minhash_dedup]
pipeline = Pipeline(datasources)
pipeline.run(global_filters=global_filters)
logger.info(f"Final dataset size: {len(pipeline.datasources[0]['dataset'])}")

[11/10/22 00:08:27] INFO     Original dataset size: 18014                                           ]8;id=117692;file:///tmp/ipykernel_218108/1617443191.py\1617443191.py]8;;\:]8;id=350765;file:///tmp/ipykernel_218108/1617443191.py#2\2]8;;\

                    INFO     Running datasource: wikitext                                          ]8;id=961561;file:///tmp/ipykernel_218108/3817924699.py\3817924699.py]8;;\:]8;id=671094;file:///tmp/ipykernel_218108/3817924699.py#25\25]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=573567;file:///tmp/ipykernel_218108/3817924699.py\3817924699.py]8;;\:]8;id=577814;file:///tmp/ipykernel_218108/3817924699.py#38\38]8;;\

                    INFO     Running filter: check_flagged_words on text                           ]8;id=910600;file:///tmp/ipykernel_218108/3817924699.py\3817924699.py]8;;\:]8;id=856234;file:///tmp/ipykernel_218108/3817924699.py#38\38]8;;\

                    INFO     Running cleaner: remove_empty_lines on text                           ]8;id=615791;file:///tmp/ipykernel_218108/3817924699.py\3817924699.py]8;;\:]8;id=255930;file:///tmp/ipykernel_218108/3817924699.py#41\41]8;;\

...

                    INFO     Running global filter: minhash_dedup                                  ]8;id=662592;file:///tmp/ipykernel_218108/3817924699.py\3817924699.py]8;;\:]8;id=870057;file:///tmp/ipykernel_218108/3817924699.py#62\62]8;;\

Indexing signatures...:   0%|          | 0/18014 [00:00<?, ?it/s]

Constructing graph...:   0%|          | 0/7757 [00:00<?, ?it/s]

Iterating over components...:   0%|          | 0/10560 [00:00<?, ?it/s]

[11/10/22 00:08:50] INFO     Final dataset size: 10560                                             ]8;id=673100;file:///tmp/ipykernel_218108/1617443191.py\1617443191.py]8;;\:]8;id=728464;file:///tmp/ipykernel_218108/1617443191.py#16\16]8;;\

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()